Inevitably, while you’re collaborating with others, you’ll find a mutual interdependency
between modules. It can even happen while you work by yourself on the various parts of a
single program.

For example, say you want your GUI application to show a dialog box for choosing where
to save a document. The data displayed by the dialog could be specified through
arguments to your event handlers. But the dialog also needs to read global state, like user
preferences, to know how to render properly.

Here, I define a dialog that retrieves the default document save location from global
preferences:


To understand what’s happening here, you need to know the details of Python’s import
machinery. When a module is imported, here’s what Python actually does in depth-first
order:
1. Searches for your module in locations from sys.path
2. Loads the code from the module and ensures that it compiles
3. Creates a corresponding empty module object
4. Inserts the module into sys.modules
5. Runs the code in the module object to define its contents

The problem with a circular dependency is that the attributes of a module aren’t defined
until the code for those attributes has executed (after step #5). But the module can be
loaded with the import statement immediately after it’s inserted into sys.modules
(after step #4).

In the example above, the app module imports dialog before defining anything. Then,
the dialog module imports app. Since app still hasn’t finished running—it’s currently
importing dialog—the app module is just an empty shell (from step #4). The
AttributeError is raised (during step #5 for dialog) because the code that defines
prefs hasn’t run yet (step #5 for app isn’t complete).

The best solution to this problem is to refactor your code so that the prefs data structure
is at the bottom of the dependency tree. Then, both app and dialog can import the same
utility module and avoid any circular dependencies. But such a clear division isn’t always
possible or could require too much refactoring to be worth the effort.

There are three other ways to break circular dependencies.


## Reordering Imports

The first approach is to change the order of imports. For example, if you import the
dialog module toward the bottom of the app module, after its contents have run, the
AttributeError goes away.

This works because, when the dialog module is loaded late, its recursive import of app
will find that app.prefs has already been defined (step #5 is mostly done for app).

Although this avoids the AttributeError, it goes against the PEP 8 style guide (see
Item 2: “Follow the PEP 8 Style Guide”). The style guide suggests that you always put
imports at the top of your Python files. This makes your module’s dependencies clear to
new readers of the code. It also ensures that any module you depend on is in scope and
available to all the code in your module.

Having imports later in a file can be brittle and can cause small changes in the ordering of
your code to break the module entirely. Thus, you should avoid import reordering to solve
your circular dependency issues.


## Import, Configure, Run

A second solution to the circular imports problem is to have your modules minimize side
effects at import time. You have your modules only define functions, classes, and
constants. You avoid actually running any functions at import time. Then, you have each
module provide a configure function that you call once all other modules have
finished importing. The purpose of configure is to prepare each module’s state by
accessing the attributes of other modules. You run configure after all modules have
been imported (step #5 is complete), so all attributes must be defined.

Here, I redefine the dialog module to only access the prefs object when configure
is called:

This works well in many situations and enables patterns like dependency injection. But
sometimes it can be difficult to structure your code so that an explicit configure step is
possible. Having two distinct phases within a module can also make your code harder to
read because it separates the definition of objects from their configuration.

## Dynamic Import
The third—and often simplest—solution to the circular imports problem is to use an
import statement within a function or method. This is called a dynamic import because
the module import happens while the program is running, not while the program is first
starting up and initializing its modules.

Here, I redefine the dialog module to use a dynamic import. The dialog.show
function imports the app module at runtime instead of the dialog module importing
app at initialization time.

This approach has a similar effect to the import, configure, and run steps from before. The
difference is that this requires no structural changes to the way the modules are defined
and imported. You’re simply delaying the circular import until the moment you must
access the other module. At that point, you can be pretty sure that all other modules have
already been initialized (step #5 is complete for everything).

In general, it’s good to avoid dynamic imports like this. The cost of the import statement
is not negligible and can be especially bad in tight loops. By delaying execution, dynamic
imports also set you up for surprising failures at runtime, such as SyntaxError
exceptions long after your program has started running (see Item 56: “Test Everything
with unittest” for how to avoid that). However, these downsides are often better than
the alternative of restructuring your entire program.

* Circular dependencies happen when two modules must call into each other at import time. They can cause your program to crash at startup.
* The best way to break a circular dependency is refactoring mutual dependencies into a separate module at the bottom of the dependency tree.
* Dynamic imports are the simplest solution for breaking a circular dependency between modules while minimizing refactoring and complexity.

